In [10]:
#前置配置

import os

file_paths = []
folder_path = '../data_base/knowledge_db'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[:3])

['../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', '../data_base/knowledge_db/prompt_engineering/9. 总结 Summary.md', '../data_base/knowledge_db/prompt_engineering/6. 文本转换 Transforming.md']


In [11]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))


In [12]:
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())


In [13]:
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")


每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'page': 1, 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20230303170709-00'00'", 'modDate': '', 'trapped': ''}
------
查看该文档的内容:
前言
“周志华老师的《机器学习》
（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”
，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。
”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”
。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”
。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(texts)


In [15]:
#构建Chroma向量库
#import os

#os.environ["OPENAI_API_KEY"] = 'sk-HVqDGdRrTm2OIqsK16F0836fE66f42E998Bc559010201c60'
#os.environ["BASE_URL"] = 'https://llm.langcore.cn/v1/'

#from langchain.embeddings.openai import OpenAIEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

embedding = ZhipuAIEmbeddings()

persist_directory = '../data_base/vector_db/chroma'



In [16]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents = split_docs[:20],
    embedding = embedding,
    persist_directory = persist_directory
)

In [18]:
vectordb.persist()

In [20]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


In [21]:
# 向量检索
# 相似度检索

question="什么是大语言模型"

sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [22]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")


检索到的第0个内容: 
3
1.4.1
式(1.1) 和式(1.2) 的解释. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
4
第2 章模型评估与选择
5
2.1
经验误差与过拟合
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
--------------
检索到的第1个内容: 
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；
• 每个公式的解析和推导我们都力(zhi) 争(neng) 以本科数学基础的视角进行讲解，所以超纲的数学知识
我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；
• 
--------------
检索到的第2个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》
←_←
目录
第1 章绪论
1
1.1
引言. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
1
1.2
基本术语
. . . . . . . . . . . . . . . . . . . .
--------------


In [23]:
# MMR检索
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)


In [24]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
3
1.4.1
式(1.1) 和式(1.2) 的解释. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
4
第2 章模型评估与选择
5
2.1
经验误差与过拟合
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
--------------
MMR 检索到的第1个内容: 
23
→_→
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
←_←
--------------
MMR 检索到的第2个内容: 
在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1 版）
最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases
编委会
主编：Sm1les、archwalker、jbb0523
编委：juxiao、Majingmin、MrBigFan、shanry、Ye
--------------
